# Part-2: MLP

Here we implement a multilayer perceptron (MLP) character-level language model using [Bengio et al. 2003 MLP language model paper.](https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)